In [3]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [4]:
inter_df = pd.read_csv('./국토교통부_건축물대장_표제부/bdinfo_baseline_maindata_inter_checkpoint_1.csv')
print(inter_df.shape)
inter_df.head()

c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(894005, 49)


,관리_건축물대장_PK,대장_구분,대장_종류,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조,기타_구조,주_용도,기타_용도,지붕,기타_지붕,높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명,기준년월_최초사용승인일,기준년월_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자
0,11440-21639,집합,표제부,0,177.62,105.300,59.28,393.130,385.210,216.87,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트 평스라브,12.00,4,1,0,0,0,0.0,393.130,0,0.0,0,0.0,0,0.00,4,46.0,20040323.0,NaN,0.0,0,NaN,0,NaN,0,20101230,201412,서울특별시 마포구 서교동 484-16,4,일반,20040323.0,20040323.0,20101230,20170209
1,11590-100182663,집합,표제부,1,389.48,232.420,59.67,947.740,659.890,169.43,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트,11.26,4,1,0,0,0,0.0,947.740,0,0.0,0,0.0,8,287.85,2,11.5,20080826.0,NaN,0.0,0,NaN,0,NaN,0,20110119,201412,서울특별시 동작구 상도동 302-28,10,일반,20080826.0,20080826.0,20110119,20190530
2,11320-100182241,집합,표제부,0,208.00,81.085,38.98,268.165,268.165,128.93,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트,10.40,4,0,0,0,0,0.0,268.165,0,0.0,0,0.0,3,0.00,1,0.0,20080721.0,NaN,0.0,0,NaN,0,NaN,0,20110202,201412,서울특별시 도봉구 쌍문동 380-24,4,일반,20080721.0,20080721.0,20110202,20190530
3,11440-32174,집합,표제부,0,314.00,180.380,57.45,627.880,627.880,199.96,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,평스라브,13.30,5,0,0,0,0,0.0,627.880,0,0.0,0,0.0,7,80.50,1,11.5,20050901.0,NaN,0.0,0,NaN,0,NaN,0,20110125,201412,서울특별시 마포구 성산동 200-260,8,일반,20050901.0,20050901.0,20110125,20171205
4,11740-100179671,집합,표제부,0,0.00,564.010,0.00,6401.190,6401.190,0.00,철근콘크리트구조,철근콘크리트구조,공동주택,공동주택(아파트),(철근)콘크리트,(철근)콘크리트,45.60,14,0,3,0,0,0.0,6401.190,0,0.0,0,0.0,0,0.00,0,0.0,20090311.0,NaN,0.0,0,NaN,0,NaN,0,20110122,201412,서울특별시 강동구 강일동 674-0,78,일반,20090311.0,20090311.0,20100918,20200428


In [3]:
inter_df['사용승인_일'] = inter_df['사용승인_일'].astype('Int64')

In [4]:
def approved_year(x):
    # x is a number
    
    if pd.isna(x) == True:
        return x
    else:
        return int(str(x)[:4])

In [5]:
inter_df['사용승인_년도'] = inter_df['사용승인_일'].apply(approved_year)
inter_df['사용승인_년도'].value_counts()

2002.0    82639
2003.0    62393
2001.0    35962
2004.0    35287
1991.0    34492
1986.0    32128
1993.0    27295
2008.0    26974
2012.0    26697
2014.0    26240
1992.0    26106
2016.0    25534
2015.0    24469
1994.0    24025
2013.0    23232
2011.0    21807
1988.0    20811
2000.0    19925
1996.0    19808
1997.0    19445
1990.0    19217
1995.0    18348
1987.0    17471
2010.0    16225
2005.0    15760
1989.0    15645
2007.0    15079
2009.0    14959
2017.0    14651
2006.0    13390
1998.0    12362
1999.0    11417
2018.0     8822
1983.0     7739
1985.0     7409
1984.0     6281
1982.0     5834
1978.0     5614
2019.0     5583
1980.0     5560
1981.0     4930
1977.0     4825
1976.0     4707
1979.0     3718
1975.0     2814
1974.0     2513
1971.0     1757
2020.0     1674
1973.0     1658
1970.0     1397
1972.0      992
1969.0      924
1968.0      696
1967.0      624
1961.0      493
1966.0      486
1965.0      365
1959.0      338
1964.0      244
1960.0      244
1963.0      175
1958.0      156
1962.0  

In [6]:
inv_df = inter_df[inter_df['사용승인_년도'] == 9920]
inv_df

,관리_건축물대장_PK,대장_구분,대장_종류,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조,기타_구조,주_용도,기타_용도,지붕,기타_지붕,높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년도,지번주소,총세대수,대장구분명,기준년도_최초사용승인일,기준년도_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자,사용승인_년도
54136,11140-2175802,집합,표제부,0,0.0,0.0,0.0,360.96,0.0,0.0,벽돌구조,"연와조, 철근콘크리트조",공동주택,다세대주택,(철근)콘크리트,스라브,0.0,3,1,0,0,0,0.0,360.96,0,0.0,0,0.0,0,0.0,0,0.0,9920130,NaN,0.0,0,NaN,0,NaN,0,20111021,201412,서울특별시 중구 신당동 409-29,6,일반,9920130.0,9920130.0,20111021,20200317,9920.0
233639,11140-21758,집합,표제부,0,0.0,0.0,0.0,360.96,0.0,0.0,벽돌구조,"연와조, 철근콘크리트조",공동주택,다세대주택,(철근)콘크리트,스라브,0.0,3,1,0,0,0,0.0,360.96,0,0.0,0,0.0,0,0.0,0,0.0,9920130,NaN,0.0,0,NaN,0,NaN,0,20111021,201612,서울특별시 중구 신당동 409-29,6,일반,9920130.0,9920130.0,20111021,20200317,9920.0
343308,11140-21758,집합,표제부,0,0.0,0.0,0.0,360.96,0.0,0.0,벽돌구조,"연와조, 철근콘크리트조",공동주택,다세대주택,(철근)콘크리트,스라브,0.0,3,1,0,0,0,0.0,360.96,0,0.0,0,0.0,0,0.0,0,0.0,9920130,NaN,0.0,0,NaN,0,NaN,0,20171101,201712,서울특별시 중구 신당동 409-29,6,일반,9920130.0,9920130.0,20111021,20200317,9920.0
452883,11140-21758,집합,표제부,0,0.0,0.0,0.0,360.96,0.0,0.0,벽돌구조,"연와조, 철근콘크리트조",공동주택,다세대주택,(철근)콘크리트,스라브,0.0,3,1,0,0,0,0.0,360.96,0,0.0,0,0.0,0,0.0,0,0.0,9920130,NaN,0.0,0,NaN,0,NaN,0,20171101,201812,서울특별시 중구 신당동 409-29,6,일반,9920130.0,9920130.0,20111021,20200317,9920.0
620255,11140-21758,집합,표제부,0,0.0,0.0,0.0,360.96,0.0,0.0,벽돌구조,"연와조, 철근콘크리트조",공동주택,다세대주택,(철근)콘크리트,스라브,0.0,3,1,0,0,0,0.0,360.96,0,0.0,0,0.0,0,0.0,0,0.0,9920130,NaN,0.0,0,NaN,0,NaN,0,20171101,201912,서울특별시 중구 신당동 409-29,6,일반,9920130.0,9920130.0,20111021,20200317,9920.0
771683,11140-21758,집합,표제부,0,0.0,0.0,0.0,360.96,0.0,0.0,벽돌구조,"연와조, 철근콘크리트조",공동주택,다세대주택,(철근)콘크리트,스라브,0.0,3,1,0,0,0,0.0,360.96,0,0.0,0,0.0,0,0.0,0,0.0,9920130,NaN,0.0,0,NaN,0,NaN,0,20111021,201512,서울특별시 중구 신당동 409-29,6,일반,9920130.0,9920130.0,20111021,20200317,9920.0


In [7]:
inter_df.loc[inv_df.index, '사용승인_일'] = 19920130

In [8]:
inter_df['사용승인_년도'] = inter_df['사용승인_일'].apply(approved_year)
inter_df['사용승인_년도'].describe()

count    889750.000000
mean       1999.794950
std          11.563201
min        1111.000000
25%        1992.000000
50%        2002.000000
75%        2008.000000
max        2020.000000
Name: 사용승인_년도, dtype: float64

In [9]:
inter_df['기준년도'].value_counts()

202011    128375
201912    127776
201712    127699
201812    127678
201612    127657
201412    127577
201512    127243
Name: 기준년도, dtype: int64

In [10]:
inter_df.to_csv('./국토교통부_건축물대장_표제부/bdinfo_baseline_maindata_inter_checkpoint_1.csv')

In [16]:
inter_df['기준년도_최초사용승인일'] = inter_df[['지번주소', '기준년도', '사용승인_일']].groupby(['지번주소', '기준년도']).transform('min')['사용승인_일']
inter_df['기준년도_최근사용승인일'] = inter_df[['지번주소', '기준년도', '사용승인_일']].groupby(['지번주소', '기준년도']).transform('max')['사용승인_일']

In [17]:
inter_df['주소별_최초생성일자'] = inter_df[['지번주소', '생성_일자']].groupby(['지번주소']).transform('min')['생성_일자']
inter_df['주소별_최근생성일자'] = inter_df[['지번주소', '생성_일자']].groupby(['지번주소']).transform('max')['생성_일자']

In [18]:
inter_df.head()

,관리_건축물대장_PK,대장_구분,대장_종류,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조,기타_구조,주_용도,기타_용도,지붕,기타_지붕,높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년도,지번주소,총세대수,대장구분명,기준년도_최초사용승인일,기준년도_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자,사용승인_년도
0,11440-2163902,집합,표제부,0,177.62,105.300,59.28,393.130,385.210,216.87,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트 평스라브,12.00,4,1,0,0,0,0.0,393.130,0,0.0,0,0.0,0,0.00,4,46.0,20040323,NaN,0.0,0,NaN,0,NaN,0,20101230,201412,서울특별시 마포구 서교동 484-16,4,일반,20040323,20040323,20101230,20170209,2004.0
1,11590-10018266302,집합,표제부,1,389.48,232.420,59.67,947.740,659.890,169.43,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트,11.26,4,1,0,0,0,0.0,947.740,0,0.0,0,0.0,8,287.85,2,11.5,20080826,NaN,0.0,0,NaN,0,NaN,0,20110119,201412,서울특별시 동작구 상도동 302-28,10,일반,20080826,20080826,20110119,20190530,2008.0
2,11320-10018224102,집합,표제부,0,208.00,81.085,38.98,268.165,268.165,128.93,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트,10.40,4,0,0,0,0,0.0,268.165,0,0.0,0,0.0,3,0.00,1,0.0,20080721,NaN,0.0,0,NaN,0,NaN,0,20110202,201412,서울특별시 도봉구 쌍문동 380-24,4,일반,20080721,20080721,20110202,20190530,2008.0
3,11440-3217402,집합,표제부,0,314.00,180.380,57.45,627.880,627.880,199.96,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,평스라브,13.30,5,0,0,0,0,0.0,627.880,0,0.0,0,0.0,7,80.50,1,11.5,20050901,NaN,0.0,0,NaN,0,NaN,0,20110125,201412,서울특별시 마포구 성산동 200-260,8,일반,20050901,20050901,20110125,20171205,2005.0
4,11740-10017967102,집합,표제부,0,0.00,564.010,0.00,6401.190,6401.190,0.00,철근콘크리트구조,철근콘크리트구조,공동주택,공동주택(아파트),(철근)콘크리트,(철근)콘크리트,45.60,14,0,3,0,0,0.0,6401.190,0,0.0,0,0.0,0,0.00,0,0.0,20090311,NaN,0.0,0,NaN,0,NaN,0,20110122,201412,서울특별시 강동구 강일동 674-0,78,일반,20090311,20090311,20100918,20200428,2009.0


In [19]:
inter_df['주소별_기준년도_총세대수합'] = inter_df[['지번주소', '기준년도', '총세대수']].groupby(['지번주소', '기준년도']).transform('sum')['총세대수']
inter_df['주소별_기준년도_총세대수합'].describe()

count    894005.000000
mean        185.387007
std         622.680007
min           0.000000
25%           7.000000
50%           9.000000
75%          19.000000
max        6920.000000
Name: 주소별_기준년도_총세대수합, dtype: float64

In [20]:
zero_df = inter_df[inter_df['주소별_기준년도_총세대수합'] == 0]
print(zero_df.shape)

(5291, 51)


In [21]:
zero_df['지번주소'].nunique()

1425

In [22]:
inter_df.to_csv('./국토교통부_건축물대장_표제부/bdinfo_baseline_maindata_inter_checpoint2.csv', index=False)

In [24]:
total_house_number_df = inter_df[['지번주소', '기준년도', '주소별_기준년도_총세대수합']].drop_duplicates().sort_values(['지번주소', '기준년도'])
print(total_house_number_df.shape)
total_house_number_df.head()

(732568, 3)


,지번주소,기준년도,주소별_기준년도_총세대수합
111184,서울특별시 강남구 개포동 1163-4,201412,20
892572,서울특별시 강남구 개포동 1163-4,201512,20
215664,서울특별시 강남구 개포동 1163-4,201612,20
310253,서울특별시 강남구 개포동 1163-4,201712,20
437662,서울특별시 강남구 개포동 1163-4,201812,20


In [25]:
total_house_number_df.to_csv('./prepped_data/기준년도별_총세대수합.csv', index=False)